In [1]:
import tensorflow as tf
from tensorflow.keras import Model

In [2]:
class SimpleModel(Model):
    
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.relu = tf.keras.layers.ReLU()
        self.fc1 = tf.keras.layers.Dense(128, use_bias=True)
        self.fc2 = tf.keras.layers.Dense(64, use_bias=True)
        self.fc3 = tf.keras.layers.Dense(10)
    
    def call(self, in_tensor):
        out = self.flatten(in_tensor)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        return self.fc3(out)

In [3]:
model = SimpleModel()

In [4]:
import tensorflow_datasets as tfds

In [5]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    download=True,
    with_info=True
)

In [6]:
def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., tf.one_hot(label, 10)

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [7]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [8]:
model.build((None, 28, 28, 1))

In [9]:
model.summary()

Model: "simple_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            multiple                  0         
_________________________________________________________________
re_lu (ReLU)                 multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  100480    
_________________________________________________________________
dense_1 (Dense)              multiple                  8256      
_________________________________________________________________
dense_2 (Dense)              multiple                  650       
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.01),
    metrics=['accuracy'],
)

In [11]:
model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
469/469 [==============================] - 2s 5ms/step - loss: 0.2252 - accuracy: 0.9299 - val_loss: 0.1455 - val_accuracy: 0.9564
Epoch 2/6
469/469 [==============================] - 1s 3ms/step - loss: 0.1241 - accuracy: 0.9627 - val_loss: 0.0981 - val_accuracy: 0.9696
Epoch 3/6
469/469 [==============================] - 1s 3ms/step - loss: 0.0944 - accuracy: 0.9714 - val_loss: 0.1358 - val_accuracy: 0.9654
Epoch 4/6
469/469 [==============================] - 1s 2ms/step - loss: 0.0861 - accuracy: 0.9744 - val_loss: 0.1028 - val_accuracy: 0.9712
Epoch 5/6
469/469 [==============================] - 1s 2ms/step - loss: 0.0807 - accuracy: 0.9765 - val_loss: 0.1723 - val_accuracy: 0.9553
Epoch 6/6
469/469 [==============================] - 1s 2ms/step - loss: 0.0744 - accuracy: 0.9782 - val_loss: 0.1151 - val_accuracy: 0.9719


In [12]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [13]:
def represent_ds():
    for _ in range(1024):
        yield [
            tf.random.uniform((1, 28, 28, 1), 0, 1, dtype=tf.float32)
        ]

In [14]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = represent_ds

In [21]:
model.save('simple_model/')

INFO:tensorflow:Assets written to: simple_model/assets


INFO:tensorflow:Assets written to: simple_model/assets


In [25]:
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

In [26]:
model_func = tf.function(lambda x: model(x))
model_func = model_func.get_concrete_function(tf.TensorSpec((None, 28, 28, 1), dtype=tf.float32))
model_func = convert_variables_to_constants_v2(model_func, lower_control_flow=False)

In [27]:
with open('simple_model/const_graph.pb', 'wb') as fid:
    fid.write(model_func.graph.as_graph_def().SerializeToString())

In [ ]:
tflite_content = converter.convert()

In [16]:
with open('./simple_model.tflite', 'wb') as fid:
    fid.write(tflite_content)